In [ ]:
import os
from google.cloud import automl

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "sa-neophilex-automl-admin.json"
project_id = "neophilex-alpha"
model_id = 'pick up the one from list models'

In [ ]:
# Lists all the files in predict folder and writes in csv file. 
#deletes first line in the file
!gsutil ls 'gs://flr_cls/predict/' > "predict.csv"
!sed -i -e "1d" predict.csv
!cat predict.csv

In [ ]:
!gsutil cp predict.csv "gs://flr_cls/csv/"

In [ ]:
input_uri = "gs://flr_cls/csv/predict.csv"
output_uri = "gs://flr_cls/csv/batch_predict_results.csv"

In [ ]:
prediction_client = automl.PredictionServiceClient()

# Get the full path of the model.
model_full_id = prediction_client.model_path(project_id, "us-central1", model_id)
gcs_source = automl.types.GcsSource(input_uris=[input_uri])

input_config = automl.types.BatchPredictInputConfig(gcs_source=gcs_source)
gcs_destination = automl.types.GcsDestination(output_uri_prefix=output_uri)
output_config = automl.types.BatchPredictOutputConfig(gcs_destination=gcs_destination)

response = prediction_client.batch_predict(model_full_id, input_config, output_config)

Note: You can use your model to make batch predictions even if your model is not deployed.

In [ ]:
print("Waiting for operation to complete...")
print("Batch Prediction results saved to Cloud Storage bucket. {}".format(response.result()))